In [3]:
import os

os.chdir('..')
os.getcwd()

'd:\\Programming\\PROJECTS\\Data Science\\FSDS\\CNN_Classifier_MLOPS'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size:list
    params_include_top:bool
    params_classes:int
    params_weights:str
    params_learning_rate:float
    
    

In [5]:
from deepclassifier.constants import *
from deepclassifier.utils import read_yaml, create_directories


In [6]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAM_FILE_PATH) -> None:
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        params=self.params
        create_directories([config.root_dir])
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_classes=params.CLASSES,
            params_image_size=params.IMAGE_SIZE,
            params_include_top=params.INCLUDE_TOP,
            params_learning_rate=params.LEARNING_RATE,
            params_weights=params.WEIGHTS,
        )
        return prepare_base_model_config


In [11]:
import tensorflow as tf


class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig) -> None:
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top,  # to have ANN after CNN or no
        )
        # https://stackoverflow.com/a/46040835
        # https://machinelearningmastery.com/how-to-use-transfer-learning-when-developing-convolutional-neural-network-models/
        # input_shape, pooling only defined when include_top=False
        self.save_model(self.config.base_model_path, self.model)

    @staticmethod
    def _prepare_full_model(
        model: tf.keras.Model,
        classes: int,
        freeze_all: bool,
        freeze_till: int,
        learning_rate: float,
    ):
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(units=classes, activation="softmax")(flatten_in)
        full_model = tf.keras.models.Model(inputs=model.input, outputs=prediction)
        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"],
        )
        full_model.summary()
        return full_model

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate,
        )
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


In [12]:
try:
    config=ConfigurationManager()
    prepare_base_model_config=config.get_prepare_base_model_config()
    prepare_base_model=PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2022-09-18 22:50:22,364] deepclassifierLogger:INFO:: common:common.py:read_yaml() => yaml file: configs\config.yaml loaded successfully
[2022-09-18 22:50:22,368] deepclassifierLogger:INFO:: common:common.py:read_yaml() => yaml file: params.yaml loaded successfully
[2022-09-18 22:50:22,373] deepclassifierLogger:INFO:: common:common.py:create_directories() => created directory at: artifacts
[2022-09-18 22:50:22,375] deepclassifierLogger:INFO:: common:common.py:create_directories() => created directory at: artifacts/prepare_base_model
[2022-09-18 22:50:22,814] tensorflow:WARNING:: saving_utils:saving_utils.py:try_build_compiled_arguments() => Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]   